In [12]:
from neo4j import GraphDatabase
import os

In [13]:
GRAPH_DB_CREDS = {
    "HOST":os.environ['NEO4J_HOST'],
    "USER" :  os.environ['NEO4J_USER'],
    "PASSWORD": os.environ['NEO4J_PASSWORD']
}

In [14]:
class Neo:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.session = self.driver.session()


    def close(self):
        self.driver.close()

    def run_read(self, fx_name, **kwargs):
        results = None
        session = self.driver.session()
        results = session.read_transaction(fx_name, **kwargs)
        return results

    def run_write(self, fx_name, **kwargs):
        results = None
        session = self.driver.session()
        results = session.write_transaction(fx_name, **kwargs)
        return results

    def run_mult(self, queries, **kwargs):
        tx = self.driver.session().begin_transaction()
        try:
            for query in queries:
                result = tx.run(query)
                print(result.data())
            tx.commit()
        except Exception as ex:
            tx.rollback()
            print("couldn't run the command")
            print(ex)
        tx.close()

    def run_query(self, query, **kwargs):   
#         data = None
        try:
            tx = self.session.begin_transaction(timeout=300)
            result = tx.run(query, **kwargs)
            data = result.to_df()
            tx.commit()
            tx.close()
            return data
        except Exception as ex:
            print("couldn't run the command")
            print(ex)
            print(query)
            return query

    
    def run_query_graph(self, query, **kwargs):   
#         data = None
        try:
            tx = self.session.begin_transaction(timeout=300)
            result = tx.run(query, **kwargs)
            return result, tx
        except Exception as ex:
            print("couldn't run the command")
            print(ex)
            print(query)
            return query
    
    def run_queries(self, queries, **kwargs):
        count = 1
        for q in queries:
            try:
                print(f"pushing company no. {count}")
                self.session.run(q)
            except Exception as ex:
                print("couldn't run the command")
                print(ex)
                print(q)
            count += 1
        print(f"Ran total queries: {len(queries)}")

In [15]:
db = Neo(GRAPH_DB_CREDS["HOST"],GRAPH_DB_CREDS["USER"], GRAPH_DB_CREDS["PASSWORD"])

In [16]:
result, tx = db.run_query_graph("MATCH (n:audio)-[r:HAS_SIMILARITY]-(n1:audio) where n.name=$name return n,r,n1 limit 50", 
                      name="120_Ben Böhmer - Autumn.mp3.wav")

In [17]:
from yfiles_jupyter_graphs import GraphWidget

In [18]:
GraphWidget(graph=result.graph())

GraphWidget(layout=Layout(height='500px', width='100%'))